In [1]:
import os
import ast

def extract_comments_from_file(filename):
    with open(filename, 'r') as file:
        tree = ast.parse(file.read(), filename=filename)

    comments = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Expr) and isinstance(node.value, ast.Str):
            comments.append(node.value.s)

    return comments

def extract_comments_from_folder(folder):
    comments = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith('.py'):
                filepath = os.path.join(root, file)
                comments.extend(extract_comments_from_file(filepath))

    return comments

folder_path = 'kattis'
comments = extract_comments_from_folder(folder_path)
for comment in comments:
    print(comment)




  Moves tiles left and merges them if possible.

  Args:
      grid: A list of lists representing the 4x4 grid.

  Returns:
      A new grid with the updated state after moving left.
  

  Moves tiles right and merges them if possible.

  Args:
      grid: A list of lists representing the 4x4 grid.

  Returns:
      A new grid with the updated state after moving right.
  

  Moves tiles up and merges them if possible.

  Args:
      grid: A list of lists representing the 4x4 grid.

  Returns:
      A new grid with the updated state after moving up.
  

  Moves tiles down and merges them if possible.

  Args:
      grid: A list of lists representing the 4x4 grid.

  Returns:
      A new grid with the updated state after moving down.
  

  Transposes the grid (swaps rows and columns).

  Args:
      grid: A list of lists representing the 4x4 grid.

  Returns:
      The transposed grid.
  

  Checks if the grid has any empty cells.

  Args:
      grid: A list of lists representing the 4x